In [ ]:
import os
os.chdir('..')
os.chdir('..')

In [ ]:
import sys

In [ ]:
pwd

In [ ]:
#%pip install tabulate

In [ ]:
from ml_grid.util import grid_param_space

In [ ]:
import ipywidgets as ipw
output = ipw.Output()

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
import os
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

In [ ]:
import ml_grid
import pathlib
import datetime
from tqdm import tqdm

import pandas as pd

from ml_grid.util.project_score_save import project_score_save_class

base_project_dir_global = 'HFE_ML_h20_experiments/'

pathlib.Path(base_project_dir_global).mkdir(parents=True, exist_ok=True) 

st_time = datetime.datetime.now().strftime("%Y-%m-%d_%I-%M-%S_%p")

base_project_dir = 'HFE_ML_h20_experiments/' + st_time + "/"
additional_naming = "HFE_ML_h20_Grid_"

print(base_project_dir)

pathlib.Path(base_project_dir).mkdir(parents=True, exist_ok=True) 

#input_csv_path = '/home/aliencat/samora/HFE/HFE/v20/30163_to_16408_imputed_outcome_grid.csv'

input_csv_path = '/home/aliencat/samora/HFE/HFE/v22/hfe_TC_merge_T_Im_10k_1yr_mean_imputed.csv'

input_csv_path = '/data/AS/Samora/HFE/HFE/v21/hfe_TC_merge_T_Im_10k_1yr_mean_imputed.csv'

input_csv_path = '/data/AS/Samora/HFE/HFE/v24 -h20/hfev_1yr_m_19072023.csv'

input_csv_path = '/data/AS/Samora/HFE/HFE/v24 -h20/hfe_TC_merge_mean_Im_10k_1yr_forward_backward_imp_M_nonan.csv'

#input_csv_path = '/data/AS/Samora/HFE/HFE/v24 -h20/hfev_1yr_5_to_1_m_19072023.csv'

#input_csv_path = 'test_data_hfe_1yr_m_small.csv' #large

#init csv to store each local projects results

project_score_save_class(base_project_dir)

n_iter = 1000

grid_iter_obj = grid_param_space.Grid(sample_n=n_iter).settings_list_iterator

for i in tqdm(range(0, n_iter)):
    if(i%2==0):
        #pass
        output.clear_output(wait=True)

    #get settings from iterator over grid of settings space
    local_param_dict = next(grid_iter_obj)

    #create object from settings
    ml_grid_object = ml_grid.pipeline.data.pipe(input_csv_path,
                                                drop_term_list=['chrom', 'hfe', 'phlebo'],
                                                local_param_dict=local_param_dict,
                                                base_project_dir = base_project_dir,
                                                additional_naming = additional_naming,
                                                test_sample_n = 0,
                                                param_space_index = i
                                                
                                                )

    from ml_grid.pipeline import main


    #pass object to be evaluated and write results to csv
    res = main.run(ml_grid_object, local_param_dict=local_param_dict).execute()
    
    

In [ ]:
print("done")

In [ ]:
break

In [ ]:
ml_grid_object.y_test.shape

In [ ]:
ml_grid_object.y_test_orig.shape

In [ ]:
.asnumeric()

In [ ]:
#%pip install scikeras

In [ ]:
main.run(ml_grid_object, local_param_dict=local_param_dict).execute()

In [ ]:
# import shutil
# shutil.rmtree('HFE_ML_h20_experiments/')

In [ ]:
column_list == column_list1

In [ ]:
#compute param grid for all then normalise... for 1:1 for each method

#apply max for under util

In [ ]:
ml_grid_object.X_train

In [ ]:
import h2o

In [ ]:
h2o.init()

In [ ]:
# Assign values to variables
X_train_data = ml_grid_object.X_train
y_train_data = ml_grid_object.y_train.to_frame()
X_test_data = ml_grid_object.X_test
y_test_data = ml_grid_object.y_test.to_frame()
X_test_orig_data = ml_grid_object.X_test_orig
y_test_orig_data = ml_grid_object.y_test_orig.to_frame()

# Create H2OFrame objects using the variables
X_train = h2o.H2OFrame(X_train_data)
y_train = h2o.H2OFrame(y_train_data)
X_test = h2o.H2OFrame(X_test_data)
y_test = h2o.H2OFrame(y_test_data)
X_test_orig = h2o.H2OFrame(X_test_orig_data)
y_test_orig = h2o.H2OFrame(y_test_orig_data)


In [ ]:
#test_h2o_frame['outcome_var_1'].asnumeric()

In [ ]:
# Assign values to variables
X_train_data = ml_grid_object.X_train
y_train_data = ml_grid_object.y_train.to_frame()
X_test_data = ml_grid_object.X_test
y_test_data = ml_grid_object.y_test.to_frame()
X_test_orig_data = ml_grid_object.X_test_orig
y_test_orig_data = ml_grid_object.y_test_orig.to_frame()

# Concatenate train and test data using Pandas
train = pd.concat([X_train_data, y_train_data], axis=1)
test = pd.concat([X_test_data, y_test_data], axis=1)
test_orig = pd.concat([X_test_orig_data, y_test_orig_data], axis=1)

train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)


outcome_variable = 'outcome_var_1'
# Identify predictors and response
x = list(train.columns)
y = outcome_variable
x.remove(y)

# For binary classification, response should be a factor
train_h2o_frame[y] = train_h2o_frame[y].asfactor()
test_h2o_frame[y] = test_h2o_frame[y].asfactor()

In [ ]:
train_h2o_frame

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=x, y=y, training_frame=train_h2o_frame)

In [ ]:
h2o.automl.get_leaderboard(aml, extra_columns = "ALL")

In [ ]:
# Get the best model from AutoML
best_model = aml.leader

# Make predictions using the best model
predictions = best_model.predict(test_h2o_frame)
print(predictions)


In [ ]:
from ml_grid.model_classes.H2OAutoMLClassifier_wrapper import H2OAutoMLClassifier

In [ ]:
H2OAutoMLClassifier().fit(train_h2o_frame, test_h2o_frame)

In [ ]:
X_train_data

In [ ]:
y_train_data.columns[0]

In [ ]:
res = H2OAutoMLClassifier().fit(X_train_data[X_train_data.columns[0:30]], y_train_data);

In [ ]:
res

In [ ]:
scores = cross_validate(
    res,
    X_train_data[X_train_data.columns[0:10]],
    y_train_data,
    scoring=metric_list,
    cv=self.cv,
    n_jobs=grid_n_jobs,  # Full CV on final best model #exp -1 was 1
    pre_dispatch = 80, #exp,
    error_score=np.nan
)

In [ ]:
res = _

In [ ]:
y_pred = res.predict(X_test_data)

In [ ]:
y_pred.shape

In [ ]:
predictions

In [ ]:
y_pred = predictions['predict'].as_data_frame().values

In [ ]:
y_pred

In [ ]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL").as_data_frame()

In [ ]:
lb

In [ ]:
lb.iloc[0]['algo']

In [ ]:
best_model.params

In [ ]:
performance = best_model.model_performance(test_h2o_frame)

In [ ]:
aml.leader

In [ ]:
from h2o.sklearn import H2OAutoMLClassifier

# Convert H2O model to Scikit-Learn model
sklearn_model = H2OAutoMLClassifier(aml.leader)


In [ ]:
sklearn_model

In [ ]:
aml.leader.explain(test_h2o_frame)

In [ ]:
type(aml.leader)

In [ ]:
aml.leader.fit(X_train_data, y_train_data)

In [ ]:
H2OStackedEnsembleEstimator 

In [ ]:
current_algorithm = sklearn_model

In [ ]:
X_train_data = ml_grid_object.X_train
y_train_data = ml_grid_object.y_train.to_frame()

In [ ]:
type(current_algorithm)

In [ ]:
current_algorithm

In [ ]:
current_algorithm.nfolds

In [ ]:
current_algorithm

In [ ]:
current_algorithm.is_classifier()

In [ ]:
current_algorithm.show()

In [ ]:
type(current_algorithm)

In [ ]:
nm = eval(str(current_algorithm.show()))

In [ ]:
nm

In [ ]:
current_algorithm.fit(X_train_data, y_train_data)
            

In [ ]:
from sklearn import metrics
from sklearn.metrics import *
from sklearn.metrics import (classification_report, f1_score, make_scorer,
                             matthews_corrcoef, roc_auc_score)
from sklearn.model_selection import (GridSearchCV, ParameterGrid,
                                     RandomizedSearchCV, RepeatedKFold,
                                     cross_validate)

In [ ]:
import numpy as np

In [ ]:
metric_list = {'auc': make_scorer(roc_auc_score, needs_proba=False),
                'f1':'f1',
                'accuracy':'accuracy',
                'recall': 'recall'}

In [ ]:
grid_n_jobs = 2

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
scores = cross_validate(
    res,
    X_train_data[X_train_data.columns[0:10]],
    y_train_data,
    scoring=metric_list,
    cv=cv,
    n_jobs=grid_n_jobs,  # Full CV on final best model #exp -1 was 1
    pre_dispatch = 80, #exp,
    error_score=np.nan
)
current_algorithm_scores = scores
#     scores_tuple_list.append((method_name, current_algorithm_scores, grid))

In [ ]:
current_algorithm_scores

In [ ]:
#best_pred_orig = current_algorithm.predict(self.X_test_orig[self.X_test_orig.columns]) #exp

#--------------------

predictions = best_model.predict(test_h2o_frame)

best_pred_orig = predictions['predict'].as_data_frame().values

In [ ]:
best_model.

In [ ]:
performance

In [ ]:
performance = best_model.model_performance(train_h2o_frame)

In [ ]:
performance

In [ ]:
my_training_frame = best_model.actual_params['training_frame']

In [ ]:
my_training_frame

In [ ]:
col_used = h2o.get_frame(my_training_frame)
col_used

In [ ]:
pwd

In [ ]:
'hfe_TC_merge_T_Im_10k_1yr_mean.csv'

In [ ]:
dd = pd.read_csv('/data/AS/Samora/HFE/HFE/v21/hfe_TC_merge_T_Im_10k_1yr_mean.csv')

In [ ]:
dd

In [ ]:
dd['client_idcode'].value_counts()

In [ ]:
dd

In [ ]:
import psutil

def get_free_space(path):
    disk_usage = psutil.disk_usage(path)
    free_space = disk_usage.free / (1024 ** 3)  # Convert bytes to gigabytes
    return free_space

device_path = "/"  # Change this to the path of the device you're interested in
free_space_gb = get_free_space(device_path)
print(f"Free space on {device_path}: {free_space_gb:.2f} GB")


In [ ]:
import psutil

def get_free_space(path):
    disk_usage = psutil.disk_usage(path)
    free_space = disk_usage.free / (1024 ** 3)  # Convert bytes to gigabytes
    return free_space

def get_all_drive_free_space():
    all_partitions = psutil.disk_partitions(all=True)
    
    for partition in all_partitions:
        device_path = partition.mountpoint
        free_space_gb = get_free_space(device_path)
        print(f"Free space on {device_path}: {free_space_gb:.2f} GB")

get_all_drive_free_space()


In [ ]:
import psutil

def get_free_space_percentage(path):
    disk_usage = psutil.disk_usage(path)
    total_capacity = disk_usage.total
    
    if total_capacity == 0:
        return 0.0  # Handle the case of zero total capacity
    
    free_space_percentage = (disk_usage.free / total_capacity) * 100
    return free_space_percentage

def get_all_drive_free_space_percentage():
    all_partitions = psutil.disk_partitions(all=True)
    
    for partition in all_partitions:
        device_path = partition.mountpoint
        free_space_percentage = get_free_space_percentage(device_path)
        print(f"Free space on {device_path}: {free_space_percentage:.2f}%")

get_all_drive_free_space_percentage()


In [ ]:
from tqdm import tqdm

In [ ]:
import os
import pandas as pd

def get_files_info(directory_path):
    files_info = []

    for root, dirs, files in tqdm(os.walk(directory_path)):
        for file in (files):
            try:
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                files_info.append({
                    'File Name': file,
                    'File Path': file_path,
                    'File Size (bytes)': file_size
                })
            except:
                pass

    return files_info

directory_path = '/data/AS/Samora'
files_info = get_files_info(directory_path)
df = pd.DataFrame(files_info)

#print(df)


In [ ]:
df

In [ ]:
df.to_csv('file_sizes.csv')

In [ ]:
df.sort_values('File Size (bytes)')

In [ ]:
df.sort_values('File Size (bytes)').tail(50
                                        )

In [ ]:
import os

def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting '{file_path}': {e}")

file_to_delete = '/data/AS/Samora/CLL/fin_Tests_controls.csv'  # Replace with the actual file path
delete_file(file_to_delete)


In [ ]:
import shutil

def delete_folder(folder_path):
    try:
        shutil.rmtree(folder_path)
        print(f"Folder '{folder_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting '{folder_path}': {e}")

folder_to_delete = '/data/AS/Samora/pregnancy_cardio/Pregnancy_documents'  # Replace with the actual folder path
delete_folder(folder_to_delete)


In [ ]:
class h2o_evaluate_class():
    
    
    def __init__(self, algorithm_implementation, parameter_space, method_name, ml_grid_object, sub_sample_parameter_val = 100): # kwargs**
        #
        
        warnings.filterwarnings('ignore') 

        warnings.filterwarnings('ignore', category=FutureWarning)

        warnings.filterwarnings('ignore', category=ConvergenceWarning)
        
        warnings.filterwarnings('ignore', category=UserWarning)
        
        h2o.init()
        
        self.global_params = global_parameters()
        
        self.verbose = self.global_params.verbose
        
        self.sub_sample_param_space_pct = self.global_params.sub_sample_param_space_pct
        
        random_grid_search = self.global_params.random_grid_search
        
        self.sub_sample_parameter_val = sub_sample_parameter_val
        
        grid_n_jobs = self.global_params.grid_n_jobs
        
        
        self.metric_list = self.global_params.metric_list
        
        self.error_raise = self.global_params.error_raise
        
        
        if(self.verbose >=3):
            print(f"crossvalidating {method_name}")
        
        self.global_parameters = global_parameters()
        
        self.ml_grid_object_iter = ml_grid_object
        
        self.X_train = h2o.H2OFrame(self.ml_grid_object_iter.X_train)
        self.y_train = h2o.H2OFrame(self.ml_grid_object_iter.y_train.to_frame())
        self.X_test = h2o.H2OFrame(self.ml_grid_object_iter.X_test)
        self.y_test = h2o.H2OFrame(self.ml_grid_object_iter.y_test.to_frame())
        self.X_test_orig = h2o.H2OFrame(self.ml_grid_object_iter.X_test_orig)
        self.y_test_orig = h2o.H2OFrame(self.ml_grid_object_iter.y_test_orig.to_frame())

        
        self.nfolds_val = -1 #auto
        
        
        # Assign values to variables
        X_train_data = ml_grid_object.X_train
        y_train_data = ml_grid_object.y_train.to_frame()
        X_test_data = ml_grid_object.X_test
        y_test_data = ml_grid_object.y_test.to_frame()
        X_test_orig_data = ml_grid_object.X_test_orig
        y_test_orig_data = ml_grid_object.y_test_orig.to_frame()

        # Concatenate train and test data using Pandas
        train = pd.concat([X_train_data, y_train_data], axis=1)
        test = pd.concat([X_test_data, y_test_data], axis=1)
        test_orig = pd.concat([X_test_orig_data, y_test_orig_data], axis=1)

        train_h2o_frame = h2o.H2OFrame(train)
        test_h2o_frame = h2o.H2OFrame(test)


        outcome_variable = 'outcome_var_1'
        
        outcome_variable = y_train_data.columns[0]
        # Identify predictors and response
        x = list(train.columns)
        y = outcome_variable
        x.remove(y)

        # For binary classification, response should be a factor
        train_h2o_frame[y] = train_h2o_frame[y].asfactor()#.asnumeric()
        test_h2o_frame[y] = test_h2o_frame[y].asfactor()#.asnumeric()
        
        
        #self.cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 
        start = time.time()

    
        aml = H2OAutoML(max_models=5, seed=1)
        aml.train(x=x, y=y, training_frame=train_h2o_frame)
        
        
        h2o.automl.get_leaderboard(aml, extra_columns = "ALL")

        if(self.global_parameters.verbose >= 4):
            
            debug_print_statements_class.debug_print_scores(scores)
            
        plot_auc = False
        if(plot_auc):    

            print(" ")
            
            
        best_model = aml.leader
            
        
        
        performance = best_model.model_performance(test_h2o_frame)
        
        
    
        #--------------------

        
        self.cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        # global scores_tuple_list
        # global i
        start = time.time()

        current_algorithm = algorithm_implementation

        parameters = parameter_space
        n_iter_v = np.nan

        sklearn_model = H2OAutoMLClassifier(aml.leader)
        
        current_algorithm = sklearn_model
        
        
        current_algorithm.fit(self.X_train, self.y_train)
            

        scores = cross_validate(
            current_algorithm,
            self.X_train,
            self.y_train,
            scoring=metric_list,
            cv=self.cv,
            n_jobs=grid_n_jobs,  # Full CV on final best model #exp -1 was 1
            pre_dispatch = 80, #exp,
            error_score=np.nan
        )
        current_algorithm_scores = scores
    #     scores_tuple_list.append((method_name, current_algorithm_scores, grid))
        

        #best_pred_orig = current_algorithm.predict(self.X_test_orig[self.X_test_orig.columns]) #exp

        #--------------------
        
        predictions = best_model.predict(test_h2o_frame)
        
        best_pred_orig = predictions['predict'].as_data_frame().values
        
        
        pg = np.nan
        

In [ ]:
import h2o
from h2o.automl import H2OAutoML
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [ ]:
# Concatenate train and test data using Pandas
train = pd.concat([X_train_data, y_train_data], axis=1)
test = pd.concat([X_test_data, y_test_data], axis=1)
test_orig = pd.concat([X_test_orig_data, y_test_orig_data], axis=1)

train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)


outcome_variable = 'outcome_var_1'
# Identify predictors and response
x = list(train.columns)
y = outcome_variable
x.remove(y)

# For binary classification, response should be a factor
train_h2o_frame[y] = train_h2o_frame[y].asfactor()
test_h2o_frame[y] = test_h2o_frame[y].asfactor()

In [ ]:
class H2OAutoMLClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_runtime_secs=60, nfolds=5, seed=1):
        self.max_runtime_secs = max_runtime_secs
        self.nfolds = nfolds
        self.seed = seed
        self.automl = None

    def fit(self, X, y):
        #X, y = check_X_y(X, y)
        self.classes_ = np.unique(y)
        outcome_var = y_train_data.columns[0]
        
        x = list(train.columns)
        y_n = outcome_var
        x.remove(y_n)
        
        h2o.init()
        #train_df = pd.concat([pd.DataFrame(X), pd.Series(y)], axis=1)
        
        train_df = pd.concat([X, y], axis=1)
        
        
        train_h2o = h2o.H2OFrame(train_df)
        
        train_h2o[y_n] = train_h2o[y_n].asfactor()
        
        self.automl = H2OAutoML(max_runtime_secs=self.max_runtime_secs, nfolds=self.nfolds, seed=self.seed)
        self.automl.train(y=y_n, x=x, training_frame=train_h2o)
        return self

    def predict(self, X):
        check_is_fitted(self)
        #X = check_array(X)
        #test_h2o = h2o.H2OFrame(pd.DataFrame(X))
        test_h2o = h2o.H2OFrame(X)
        predictions = self.automl.leader.predict(test_h2o)
        
        return predictions[:,0]
        #return predictions.as_data_frame().values

    def predict_proba(self, X):
        raise NotImplementedError("H2O AutoML does not support predict_proba.")

    def get_params(self, deep=True):
        return {
            "max_runtime_secs": self.max_runtime_secs,
            "nfolds": self.nfolds,
            "seed": self.seed,
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self


In [ ]:
X_train_data

In [ ]:
h2o_automl = H2OAutoMLClassifier(max_runtime_secs=10)
h2o_automl.fit(X_train_data, y_train_data)

In [ ]:
X_test_data

In [ ]:
y_pred = h2o_automl.predict(X_train_data)

In [ ]:
y_pred[:,0]

In [ ]:
# Make predictions
y_pred = h2o_automl.predict(X_test_data)

# Calculate accuracy
accuracy = accuracy_score(y_test_data, y_pred)